# Podcast Automation Pipeline
This Colab notebook automates steps 2–4:
1. Video candidate search (Step 2)
2. Download & Whisper transcription (Step 3)
3. GPT summarization & script generation (Step 4)

## 1. Environment Setup

In [5]:
# ========== セル 1: 環境設定 ==========
# 目的: Google DriveをColabに接続し、必要なPythonライブラリをインストールします。

# Google DriveをColabから使えるようにします。
from google.colab import drive
drive.mount('/content/drive') # '/content/drive' という場所にDriveが接続されます

# 必要なPythonライブラリをインストール・アップグレードします。
# yt-dlp: YouTube動画をダウンロードするため
# openai-whisper: 音声を文字起こしするため
# google-api-python-client: YouTube APIを使うため
# python-dotenv: .envファイルから環境変数を読み込むため (今回はColabシークレット優先なので必須ではない)
# google-generativeai: Gemini APIを使うため
!pip install --upgrade yt-dlp openai-whisper google-api-python-client python-dotenv google-generativeai

# ffmpegという音声・動画処理ツールをインストールします (yt-dlpやWhisperで必要になることがあります)。
!apt-get update && apt-get install -y ffmpeg

print("✅ 環境設定セルが完了しました。")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 3,632 B in 2s (2,346 B/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2

## 2. Imports and Constants

In [6]:
# ========== セル 2: Imports and Constants (主要な準備) ==========
# 目的: このノートブック全体で使う主要なライブラリのインポート、
#       重要な変数(ファイルパスなど)の定義、APIクライアントやAIモデルの準備を行います。

print("▶️ Imports and Constants セルを開始します...")

# ─── 1. ライブラリのインポート ───
import os # ファイルパス操作などOS機能を使うため
import pandas as pd # ExcelやCSVデータを扱うため (Pandas DataFrame)
import subprocess # 外部コマンド(yt-dlpなど)を実行するため
import whisper    # 音声認識ライブラリWhisperを使うため
from google.colab import drive # Drive操作 (このセルでは主にパス設定で利用)
from google.colab import userdata # Colabのシークレット機能からAPIキーなどを安全に読み込むため
from googleapiclient.discovery import build # YouTube Data APIのクライアントを作るため
import google.generativeai as genai # Gemini APIライブラリ
import html # HTML特殊文字をエスケープするため (SSML生成時に役立つ)

print("   ✅ 主要ライブラリをインポートしました。")

# ─── 2. Google DriveのDRIVE_ROOT設定 ───
# このプロジェクトのファイルが保存されているGoogle Driveの基本フォルダパスを指定します。
# このパスは、あなたの実際のフォルダ構成に合わせてください。
DRIVE_ROOT = "/content/drive/MyDrive/Colab Notebooks/00_Podcast/yt_podcast"
if not os.path.isdir(DRIVE_ROOT):
    print(f"‼️エラー: DRIVE_ROOT に指定されたパスが見つかりません: {DRIVE_ROOT}")
    print("   Google Driveが正しくマウントされているか、上記のパスが正しいか確認してください。")
    DRIVE_ROOT = None # エラーの場合はNoneとし、後続処理で適切に扱えるようにする
else:
    print(f"   ✅ Google Drive ルートパス設定完了。DRIVE_ROOT: {DRIVE_ROOT}")

# ─── 3. APIキーの読み込み (Colabシークレットから) ───
# Colabのシークレット機能に 'YT_Gemini_Key' という名前で保存した共通APIキーを読み込みます。
# このキーはYouTube APIとGemini APIの両方で使います。
COMMON_API_KEY_NAME_IN_SECRETS = 'YT_Gemini_Key' # あなたがシークレットに設定した名前に合わせる
API_KEY_STRING = userdata.get(COMMON_API_KEY_NAME_IN_SECRETS)

YT_KEY = None # YouTube APIキー用変数
GOOGLE_AI_API_KEY = None # Gemini APIキー用変数

if not API_KEY_STRING:
    print(f"‼️エラー: APIキーがColabシークレット「{COMMON_API_KEY_NAME_IN_SECRETS}」に設定されていません。")
    print(f"   左パネルの鍵アイコンからシークレット名「{COMMON_API_KEY_NAME_IN_SECRETS}」でAPIキーを設定してください。")
else:
    YT_KEY = API_KEY_STRING
    GOOGLE_AI_API_KEY = API_KEY_STRING
    print(f"   ✅ APIキーをColabシークレット「{COMMON_API_KEY_NAME_IN_SECRETS}」から読み込みました。")

# ─── 4. YouTube API クライアント初期化 ───
yt = None # YouTube APIクライアントを格納する変数
if YT_KEY: # APIキーが読み込めていれば実行
    try:
        yt = build("youtube", "v3", developerKey=YT_KEY)
        print("   ✅ YouTube APIクライアント (yt) を作成しました。")
    except Exception as e:
        print(f"‼️エラー: YouTube APIクライアント (yt) の作成に失敗しました: {e}")
else:
    print("   ⚠️ YouTube APIキーがないため、APIクライアント (yt) は作成されませんでした。")

# ─── 5. Gemini API 設定 ───
gemini_api_configured = False # Gemini APIが設定できたかのフラグ
if GOOGLE_AI_API_KEY: # APIキーが読み込めていれば実行
    try:
        genai.configure(api_key=GOOGLE_AI_API_KEY)
        print("   ✅ Gemini API を設定しました。")
        gemini_api_configured = True
    except Exception as e:
        print(f"‼️エラー: Gemini API の設定に失敗しました: {e}")
else:
    print("   ⚠️ Gemini APIキーがないため、Gemini API は設定されませんでした。")

# ─── 6. Whisperモデル初期化 ───
whisper_model = None # Whisperモデルを格納する変数
try:
    whisper_model = whisper.load_model("base") # "base"モデルをロード (他に "small", "medium" なども選択可)
    print("   ✅ Whisperモデル (base) をロードしました。")
except NameError: # 'whisper'自体が未定義(インポート忘れ)の場合
    print(f"‼️エラー: Whisperモデルのロードに失敗しました。'whisper' ライブラリがインポートされていない可能性があります。")
    print("   セルの一番最初で 'import whisper' が実行されているか確認してください。")
except Exception as e: # その他のロードエラー
    print(f"‼️エラー: Whisperモデルのロード中に予期せぬエラーが発生しました: {e}")

# ─── 7. 主要ディレクトリパス定義と作成 ───
# 各処理ステップで使うフォルダのパスを定義し、もし存在しなければ作成します。
MP3_DIR = None
TRANS_DIR = None
SCRIPTS_DIR = None
OUTPUTS_DIR = None # STEP5 (音声合成) で生成されるMP3を保存するフォルダ

if DRIVE_ROOT: # DRIVE_ROOTが正しく設定されていれば実行
    MP3_DIR     = os.path.join(DRIVE_ROOT, "mp3s")         # ダウンロードした音声(mp3)保存場所
    TRANS_DIR   = os.path.join(DRIVE_ROOT, "transcripts")  # 文字起こし結果(txt)保存場所
    SCRIPTS_DIR = os.path.join(DRIVE_ROOT, "scripts")      # 生成された台本(txt)保存場所
    OUTPUTS_DIR = os.path.join(DRIVE_ROOT, "outputs")      # 最終的なポッドキャスト音声(mp3)保存場所

    try:
        for d_path in (MP3_DIR, TRANS_DIR, SCRIPTS_DIR, OUTPUTS_DIR):
            if d_path: # パスがNoneでなければフォルダ作成
                 os.makedirs(d_path, exist_ok=True) # exist_ok=True でフォルダが既に在ってもエラーにしない
        print(f"   ✅ 出力用ディレクトリ (mp3s, transcripts, scripts, outputs) を確認/作成しました。")
    except Exception as e:
        print(f"‼️エラー: 出力用ディレクトリの作成中にエラーが発生しました: {e}")
else:
    print(f"   ⚠️ DRIVE_ROOT が無効なため、出力用ディレクトリは設定/作成されませんでした。")

# ─── 8. 動画検索関数定義 (search_videos) ───
# STEP 2-2 (動画検索) で使う関数をここで定義しておきます。
def search_videos(youtube_client, keyword, lang, max_results=10,
                  videoDuration="medium", publishedAfter="2022-01-01T00:00:00Z"):
    """
    指定された条件でYouTube動画を検索し、結果のリストを返します。
    引数:
        youtube_client: 初期化済みのYouTube APIクライアント
        keyword (str): 検索キーワード
        lang (str): 検索結果の言語 (例: "en", "es", "ja")
        max_results (int): 最大取得件数
        videoDuration (str): 動画の長さ ("short", "medium", "long")
        publishedAfter (str): この日付以降に公開された動画を検索 (YYYY-MM-DDTHH:MM:SSZ形式)
    戻り値:
        list: 動画情報の辞書が含まれたリスト。エラー時は空リスト。
    """
    if not youtube_client: # APIクライアントが準備できていなければ処理しない
        print(f"   ⚠️ 動画検索スキップ (search_videos): YouTube APIクライアントが利用できません (キーワード: {keyword})。")
        return []
    try:
        # 実際にAPIを呼び出す前に、どんな条件で検索するかログに出します。
        print(f"      🔎 API検索実行: '{keyword}' ({lang}), Max: {max_results}, Duration: {videoDuration}, After: {publishedAfter[:10]}")

        # YouTube Data APIのsearch.listメソッドを呼び出します。
        response = youtube_client.search().list(
            part="snippet",             # 動画のタイトルや説明などの基本情報を取得
            q=keyword,                  # 検索キーワード
            type="video",               # 動画のみを対象
            relevanceLanguage=lang,     # 指定言語への関連性が高いものを優先
            videoDuration=videoDuration,# 動画の長さ
            videoLicense="creativeCommon", # クリエイティブ・コモンズライセンスの動画 (再利用しやすい)
            publishedAfter=publishedAfter, # 公開日時のフィルタ
            maxResults=max_results      # 最大取得件数
        ).execute() # APIリクエストを実行

        found_videos = [] # 見つかった動画情報を格納するリスト
        for item in response.get("items", []): # 結果の各アイテムを処理
            video_id = item["id"]["videoId"]
            snippet = item["snippet"]
            found_videos.append({
                "source": "YouTube",
                "lang": lang, # どの言語で検索したかの情報も保存
                "keyword": keyword,
                "videoId": video_id,
                "title": snippet["title"],
                "url": f"https://youtu.be/{video_id}", # 動画URL
                "publishedAt": snippet["publishedAt"]  # APIからの実際の公開日時
            })
        print(f"      🔍 API検索結果: 「{keyword}」({lang}) で {len(found_videos)} 件の動画が見つかりました。")
        return found_videos
    except Exception as e: # API呼び出し中や結果処理中にエラーが起きた場合
        print(f"   ‼️エラー (search_videos): 動画検索中にエラーが発生しました。")
        print(f"      Keyword: {keyword}, Lang: {lang}, Params: Duration={videoDuration}, After={publishedAfter}")
        print(f"      Error details: {e}")
        return [] # エラー時は空のリストを返す

print("✅ Imports and Constants セル実行完了。すべての準備が整いました。")

▶️ Imports and Constants セルを開始します...
   ✅ 主要ライブラリをインポートしました。
   ✅ Google Drive ルートパス設定完了。DRIVE_ROOT: /content/drive/MyDrive/Colab Notebooks/00_Podcast/yt_podcast
   ✅ APIキーをColabシークレット「YT_Gemini_Key」から読み込みました。
   ✅ YouTube APIクライアント (yt) を作成しました。
   ✅ Gemini API を設定しました。
   ✅ Whisperモデル (base) をロードしました。
   ✅ 出力用ディレクトリ (mp3s, transcripts, scripts, outputs) を確認/作成しました。
✅ Imports and Constants セル実行完了。すべての準備が整いました。


## 3. Step 2: Video Candidate Search

STEP2-1

In [7]:
# ========== セル 3: STEP 2-1 (検索条件のExcelからの読み込み) ==========
# 目的: "Search_list.xlsx" というExcelファイルから検索キーワードや条件を読み込みます。
#       このExcelファイルは事前に DRIVE_ROOT に配置しておく必要があります。

print("▶️ STEP 2-1: 検索条件の読み込みを開始します...")

excel_file_name = "Search_list.xlsx" # 読み込むExcelファイル名
search_queries_from_excel = [] # Excelから読み込んだ検索条件を格納するリスト
excel_file_path = None # Excelファイルのフルパス

# デフォルト値 (Excelに対応する列がない場合や値が空の場合に使用)
DEFAULT_VIDEO_DURATION = "medium"
DEFAULT_PUBLISHED_AFTER_YEAR = 2022 # デフォルトの公開年

# DRIVE_ROOTが正しく設定されているか、Excelファイルが存在するかを確認
if not DRIVE_ROOT:
    print(f"‼️エラー: 定数 DRIVE_ROOT が設定されていません。Excelファイルを読み込めません。")
else:
    excel_file_path = os.path.join(DRIVE_ROOT, excel_file_name)
    if not os.path.exists(excel_file_path):
        print(f"⚠️ Excelファイルが見つかりません: {excel_file_path}")
        print("   このステップはスキップされます。手動で検索条件リストを作成するか、Excelファイルを配置してください。")
    else:
        try:
            # Pandasを使ってExcelファイルを読み込みます
            df_search_list = pd.read_excel(excel_file_path)
            print(f"   ℹ️ Excelファイル ({excel_file_path}) を読み込みました。内容を処理します...")

            for index, row in df_search_list.iterrows(): # Excelの各行を処理
                keyword_excel = str(row.get("keyword", "")).strip() # keyword列の値を取得 (なければ空文字)
                if not keyword_excel: # キーワードが空ならその行はスキップ
                    print(f"   ℹ️ Excelの {index+2} 行目: keywordが空のためスキップします。")
                    continue

                # 各列の値を読み込み、なければデフォルト値を使用
                lang_excel = str(row.get("lang", "en")).strip().lower()
                max_results_excel = int(row.get("max_results", 1)) # デフォルト1件

                video_duration_excel = str(row.get("videoDuration", DEFAULT_VIDEO_DURATION)).strip().lower()
                if video_duration_excel not in ["short", "medium", "long"]: # 有効な値かチェック
                    print(f"   ⚠️ Excelの {index+2} 行目 ({keyword_excel}): videoDurationが無効な値「{video_duration_excel}」。デフォルト「{DEFAULT_VIDEO_DURATION}」を使用。")
                    video_duration_excel = DEFAULT_VIDEO_DURATION

                published_year_excel_str = str(row.get("publishedAfterYear", "")).strip()
                published_after_formatted = f"{DEFAULT_PUBLISHED_AFTER_YEAR}-01-01T00:00:00Z" # デフォルト
                if published_year_excel_str:
                    try:
                        year = int(float(published_year_excel_str)) # 文字列を整数に変換
                        if 1970 <= year <= 2100: # 妥当な年の範囲かチェック
                           published_after_formatted = f"{year}-01-01T00:00:00Z"
                        else:
                            print(f"   ⚠️ Excelの {index+2} 行目 ({keyword_excel}): publishedAfterYear「{year}」が範囲外。デフォルト「{DEFAULT_PUBLISHED_AFTER_YEAR}年」を使用。")
                    except ValueError: # 数字に変換できない場合
                        print(f"   ⚠️ Excelの {index+2} 行目 ({keyword_excel}): publishedAfterYear「{published_year_excel_str}」が数値でない。デフォルト「{DEFAULT_PUBLISHED_AFTER_YEAR}年」を使用。")
                else: # publishedAfterYearが空欄の場合
                     print(f"   ℹ️ Excelの {index+2} 行目 ({keyword_excel}): publishedAfterYearが空。デフォルト「{DEFAULT_PUBLISHED_AFTER_YEAR}年」を使用。")

                # 処理した検索条件をリストに追加
                search_queries_from_excel.append({
                    "keyword": keyword_excel,
                    "lang": lang_excel,
                    "max_results": max_results_excel,
                    "videoDuration": video_duration_excel,
                    "publishedAfter": published_after_formatted
                })

            if search_queries_from_excel:
                print(f"✅ {len(search_queries_from_excel)} 件の検索条件をExcelから読み込みました。")
                print("   読み込まれた検索条件の概要:")
                for i, q_item in enumerate(search_queries_from_excel):
                    print(f"     {i+1}. Keyword: '{q_item['keyword']}', Lang: '{q_item['lang']}', Max: {q_item['max_results']}, Duration: '{q_item['videoDuration']}', After: '{q_item['publishedAfter'][:4]}'")
            else:
                print(f"⚠️ Excelファイルを読み込みましたが、有効な検索条件が見つかりませんでした（キーワードが空など）。")
        except Exception as e:
            print(f"‼️エラー: Excelファイル ({excel_file_path}) の読み込みまたは処理中に失敗しました: {e}")
            search_queries_from_excel = [] # エラー時はリストを空にする

if not search_queries_from_excel:
    print("‼️STEP 2-1 完了 (警告): 有効な検索条件が読み込まれませんでした。以降の動画検索は行われません。")
else:
    print("✅ STEP 2-1: 検索条件の読み込みが完了しました。")

▶️ STEP 2-1: 検索条件の読み込みを開始します...
   ℹ️ Excelファイル (/content/drive/MyDrive/Colab Notebooks/00_Podcast/yt_podcast/Search_list.xlsx) を読み込みました。内容を処理します...
✅ 8 件の検索条件をExcelから読み込みました。
   読み込まれた検索条件の概要:
     1. Keyword: 'business bestsellers', Lang: 'en', Max: 1, Duration: 'medium', After: '2023'
     2. Keyword: 'top business books insights', Lang: 'en', Max: 1, Duration: 'medium', After: '2022'
     3. Keyword: 'famous entrepreneurs interview', Lang: 'en', Max: 1, Duration: 'long', After: '2022'
     4. Keyword: 'success havits', Lang: 'en', Max: 1, Duration: 'short', After: '2022'
     5. Keyword: 'productivity hacks for professionals', Lang: 'en', Max: 1, Duration: 'short', After: '2022'
     6. Keyword: 'resumen libros de negocios', Lang: 'es', Max: 1, Duration: 'short', After: '2022'
     7. Keyword: 'entrevistas emprendedores exitosos', Lang: 'es', Max: 1, Duration: 'short', After: '2022'
     8. Keyword: 'hábitos de éxito', Lang: 'es', Max: 1, Duration: 'short', After: '2022'
✅ STEP 2-1

STEP2-2

In [8]:
# ========== セル 4: STEP 2-2 (動画検索の実行) ==========
# 目的: STEP 2-1でExcelから読み込んだ検索条件リスト (search_queries_from_excel) を使って、
#       実際にYouTube動画を検索し、結果をCSVファイルに保存します。

print("\n▶️ STEP 2-2: 動画検索 & ID収集を開始します...")

all_found_videos = [] # 検索で見つかった全ての動画情報を格納するリスト

# 前のステップで必要な変数が準備できているか確認
if 'search_queries_from_excel' not in globals() or not isinstance(search_queries_from_excel, list):
    print("‼️エラー: STEP 2-1で検索条件リスト (search_queries_from_excel) が正しく生成されていません。")
elif not search_queries_from_excel: # 検索条件が0件の場合
    print("ℹ️ STEP 2-1で読み込まれた検索条件が0件だったため、動画検索は行われません。")
elif 'yt' not in globals() or yt is None: # YouTube APIクライアントがない場合
    print("⚠️ YouTube APIクライアントが利用できないため、動画検索は実行されませんでした。「Imports and Constants」セルを確認してください。")
else:
    print(f"   ▶️ {len(search_queries_from_excel)} 件の検索条件に基づいて動画を検索します...")
    for search_query_item in search_queries_from_excel: # 各検索条件でループ
        # search_videos関数 (「Imports and Constants」セルで定義済み) を呼び出す
        video_results_list = search_videos(
            youtube_client=yt, # YouTube APIクライアント
            keyword=search_query_item["keyword"],
            lang=search_query_item["lang"],
            max_results=search_query_item["max_results"],
            videoDuration=search_query_item["videoDuration"],
            publishedAfter=search_query_item["publishedAfter"]
        )
        all_found_videos.extend(video_results_list) # 見つかった動画を全体のリストに追加
        print("-" * 30) # 各検索条件の区切り線

    if not all_found_videos and search_queries_from_excel: # 検索はしたが結果0件
         print("ℹ️ 動画検索は実行されましたが、条件に一致する動画は見つかりませんでした。")
    elif all_found_videos: # 何か動画が見つかった
        print(f"✅ 全ての検索条件での動画検索が完了しました。合計 {len(all_found_videos)} 件の候補が見つかりました。")

# 見つかった動画情報をPandas DataFrameに変換
df_candidates = pd.DataFrame(all_found_videos)

# ─── CSVファイルへの保存 ───
if not df_candidates.empty: # DataFrameが空でなければ保存
    if DRIVE_ROOT and MP3_DIR: # DRIVE_ROOTなどが正しく設定されていれば
        candidates_csv_path = os.path.join(DRIVE_ROOT, "yt_candidates.csv")
        try:
            df_candidates.to_csv(candidates_csv_path, index=False, encoding="utf-8-sig") # index=FalseでDataFrameのインデックスは保存しない
            print(f"✅ STEP 2-2完了！ 動画候補リストをCSVに保存しました: {candidates_csv_path}")
        except Exception as e:
            print(f"‼️エラー: 動画候補リストのCSVファイルへの保存に失敗しました: {e}")
    else:
        print("⚠️ CSVファイルは保存されませんでした (DRIVE_ROOT が正しく設定されていないか、存在しないディレクトリです)。")
elif ('search_queries_from_excel' in globals() and search_queries_from_excel and
      'yt' in globals() and yt and not all_found_videos): # 検索はしたが結果0件の場合
    print("ℹ️ 検索結果が0件だったため、yt_candidates.csv は作成されませんでした。")

# ─── 結果のプレビュー ───
if not df_candidates.empty:
    print("\n📄 検索結果プレビュー (最初の5件):")
    print(df_candidates.head())
else: # DataFrameが空の場合
    if ('search_queries_from_excel' in globals() and search_queries_from_excel and
        'yt' in globals() and yt): # 検索が試みられたが結果0件
        print("\nℹ️ 表示する検索結果がありません (0件でした)。")

# セルの最終的な完了メッセージ
if 'search_queries_from_excel' not in globals() or not search_queries_from_excel:
     print("✅ STEP 2-2 完了 (注意): 検索条件がなかったため、実質的な動画検索処理は行われませんでした。")
elif not df_candidates.empty:
    pass # 正常完了のメッセージはCSV保存時に出力済み
else:
    print("✅ STEP 2-2 完了 (注意): 動画は見つかりませんでしたが、検索処理は終了しました。")


▶️ STEP 2-2: 動画検索 & ID収集を開始します...
   ▶️ 8 件の検索条件に基づいて動画を検索します...
      🔎 API検索実行: 'business bestsellers' (en), Max: 1, Duration: medium, After: 2023-01-01
      🔍 API検索結果: 「business bestsellers」(en) で 1 件の動画が見つかりました。
------------------------------
      🔎 API検索実行: 'top business books insights' (en), Max: 1, Duration: medium, After: 2022-01-01
      🔍 API検索結果: 「top business books insights」(en) で 1 件の動画が見つかりました。
------------------------------
      🔎 API検索実行: 'famous entrepreneurs interview' (en), Max: 1, Duration: long, After: 2022-01-01
      🔍 API検索結果: 「famous entrepreneurs interview」(en) で 1 件の動画が見つかりました。
------------------------------
      🔎 API検索実行: 'success havits' (en), Max: 1, Duration: short, After: 2022-01-01
      🔍 API検索結果: 「success havits」(en) で 1 件の動画が見つかりました。
------------------------------
      🔎 API検索実行: 'productivity hacks for professionals' (en), Max: 1, Duration: short, After: 2022-01-01
      🔍 API検索結果: 「productivity hacks for professionals」(en) で 1 件の動画が見つかりました。


## 4. Step 3: Download & Transcription

テスト実行

In [9]:
# ========== セル 5: STEP 3-1 (準備: CSV読み込みとダウンロード＆文字起こし関数定義) ==========
# 目的: STEP 2で作成した動画候補リスト(yt_candidates.csv)を読み込み、
#       個々の動画をダウンロードして文字起こしする関数 (download_and_transcribe) を定義します。

print("▶️ STEP 3-1: CSV読み込みとダウンロード＆文字起こし関数の準備を開始します...")

# --- 必要なグローバル変数が「Imports and Constants」セルで定義されているか確認 ---
required_globals_step3 = ['DRIVE_ROOT', 'MP3_DIR', 'TRANS_DIR', 'whisper_model']
missing_globals_step3 = [var for var in required_globals_step3 if var not in globals() or globals()[var] is None]
if missing_globals_step3:
    print(f"‼️エラー: STEP3に必要なグローバル変数が未定義またはNoneです: {', '.join(missing_globals_step3)}")
    print("   「Imports and Constants」セルが正しく実行されているか確認してください。")
    # この後の処理でエラーになる可能性が高いため、ここで処理を止めることも検討
    # exit()

# ① STEP2で作成したCSVファイルを読み込みます
csv_candidates_file_name = "yt_candidates.csv"
candidates_csv_full_path = None
df_step3_videos = pd.DataFrame() # 処理対象の動画リストを格納するDataFrame

if DRIVE_ROOT and os.path.isdir(DRIVE_ROOT): # DRIVE_ROOTが有効か確認
    candidates_csv_full_path = os.path.join(DRIVE_ROOT, csv_candidates_file_name)
    if os.path.exists(candidates_csv_full_path):
        try:
            df_step3_videos = pd.read_csv(candidates_csv_full_path)
            if df_step3_videos.empty:
                print(f"   ⚠️ {csv_candidates_file_name} は空です。処理対象の動画がありません。")
            else:
                print(f"   ✅ {csv_candidates_file_name} を読み込みました。{len(df_step3_videos)} 件の動画候補があります。")
        except Exception as e:
            print(f"   ‼️エラー: {csv_candidates_file_name} の読み込みに失敗しました: {e}")
    else: # CSVファイルが見つからない場合
        print(f"   ⚠️ {csv_candidates_file_name} が見つかりません: {candidates_csv_full_path}")
else: # DRIVE_ROOTが無効な場合
    print(f"   ⚠️ DRIVE_ROOT が無効なため、{csv_candidates_file_name} を読み込めません。")


# ② １本分の動画をダウンロードして文字起こしする関数
def download_and_transcribe(video_info_row, target_mp3_dir, target_transcript_dir, audio_model):
    """
    与えられた動画情報(Pandas Series)に基づいて、音声をダウンロードし、文字起こしを行います。
    引数:
        video_info_row (pd.Series): 'url', 'videoId', 'lang' 列を含む動画情報
        target_mp3_dir (str): MP3ファイルの保存先ディレクトリパス
        target_transcript_dir (str): 文字起こしテキストの保存先ディレクトリパス
        audio_model: 初期化済みのWhisperモデルオブジェクト
    戻り値:
        str: 処理が成功した場合(またはスキップした場合)は video_id。失敗時は None。
    """
    # 引数や必要なディレクトリ/モデルが正しく渡されているか基本的なチェック
    if not all([target_mp3_dir, target_transcript_dir, audio_model is not None]):
        print("   ‼️エラー (download_and_transcribe): 保存先ディレクトリまたはWhisperモデルが正しく設定されていません。")
        return None

    video_url = video_info_row.get("url")
    if not video_url: # URLがなければ処理できない
        print("   ‼️エラー (download_and_transcribe): 動画情報に 'url' が含まれていません。")
        return None

    # videoId を取得 (CSVにあればそれを使い、なければURLから抽出試行)
    video_id_str = None
    if "videoId" in video_info_row and pd.notna(video_info_row["videoId"]):
        video_id_str = str(video_info_row["videoId"])
    elif "/" in video_url: # URLから末尾部分を取得
        video_id_str = video_url.rsplit("/",1)[-1]
        if "=" in video_id_str: # "watch?v=VIDEO_ID" のような形式から抽出
            video_id_str = video_id_str.split("=")[-1].split("&")[0]
    if not video_id_str:
        print(f"   ‼️エラー (download_and_transcribe): URL '{video_url}' から videoId を抽出できませんでした。")
        return None

    # 文字起こしに使用する言語を取得 (なければデフォルト 'en')
    transcription_lang = str(video_info_row.get("lang", "en")).strip().lower()

    # 保存するファイルパスを生成
    mp3_file_path = os.path.join(target_mp3_dir, f"{video_id_str}.mp3")
    transcript_file_path = os.path.join(target_transcript_dir, f"{video_id_str}.txt")
    processed_successfully_vid = None # 処理成功フラグ

    try:
        # 1) 音声(MP3)ダウンロード (既にファイルがあればスキップ)
        if not os.path.exists(mp3_file_path):
            print(f"      ▶️ ダウンロード開始: {video_id_str}.mp3 (URL: {video_url})")
            # yt-dlpコマンドを実行して音声をダウンロード
            # check=True: エラー時に例外を発生させる
            # capture_output=True, text=True: コマンドの出力を取得する
            # timeout: コマンド実行のタイムアウト時間(秒)
            dl_result = subprocess.run([
                "yt-dlp",
                "-x", "--audio-format", "mp3", # mp3形式で音声のみ抽出
                "--no-playlist",             # プレイリストの場合は個々の動画のみ対象
                "--socket-timeout", "30",    # 通信タイムアウト
                "-o", mp3_file_path,         # 出力ファイルパス
                video_url
            ], check=True, capture_output=True, text=True, timeout=180) # ダウンロード全体のタイムアウトを少し長めに
            print(f"      ✅ ダウンロード完了: {video_id_str}.mp3")
        else:
            print(f"      ℹ️ ダウンロードスキップ: {video_id_str}.mp3 は既に存在します。")

        # 2) Whisper 文字起こし (既にファイルがあればスキップ)
        if not os.path.exists(transcript_file_path):
            if not os.path.exists(mp3_file_path): # MP3ファイルがなければ文字起こしできない
                print(f"      ‼️エラー (文字起こし): MP3ファイル ({mp3_file_path}) が見つかりません。ダウンロード失敗の可能性。")
            else:
                print(f"      ▶️ 文字起こし開始: {video_id_str}.txt (言語: {transcription_lang if transcription_lang else '自動検出'})")
                # Whisperモデルを使って文字起こし実行
                transcription_output = audio_model.transcribe(mp3_file_path, language=transcription_lang if transcription_lang else None)
                with open(transcript_file_path, "w", encoding="utf-8") as f:
                    f.write(transcription_output["text"])
                print(f"      ✅ 文字起こし完了: {video_id_str}.txt")
        else:
            print(f"      ℹ️ 文字起こしスキップ: {video_id_str}.txt は既に存在します。")

        processed_successfully_vid = video_id_str # ここまでくればOK

    except subprocess.CalledProcessError as e_dl: # yt-dlp実行エラー
        print(f"      ‼️エラー (yt-dlp): {video_id_str} のダウンロードに失敗しました。")
        print(f"         コマンド: {' '.join(e_dl.cmd)}")
        print(f"         Returncode: {e_dl.returncode}")
        if e_dl.stderr: print(f"         Stderr: {e_dl.stderr.strip()}")
        if os.path.exists(mp3_file_path): # もし不完全なMP3ファイルができていたら削除
            try: os.remove(mp3_file_path)
            except OSError: pass
    except Exception as e_main: # その他の予期せぬエラー
        print(f"      ‼️エラー (download_and_transcribe): {video_id_str} の処理中に予期せぬエラー: {e_main}")

    return processed_successfully_vid

# セルの完了メッセージ
if df_step3_videos.empty and not (DRIVE_ROOT and os.path.exists(os.path.join(DRIVE_ROOT, csv_candidates_file_name))):
    print("✅ STEP 3-1 完了 (警告): 処理対象のCSVファイルが見つからないか空でした。")
elif df_step3_videos.empty:
    print("✅ STEP 3-1 完了 (警告): 処理対象の動画データがありません (CSVは存在したが空)。")
else:
    print("✅ STEP 3-1 完了: CSV読み込みと `download_and_transcribe` 関数の準備ができました。")

▶️ STEP 3-1: CSV読み込みとダウンロード＆文字起こし関数の準備を開始します...
   ✅ yt_candidates.csv を読み込みました。8 件の動画候補があります。
✅ STEP 3-1 完了: CSV読み込みと `download_and_transcribe` 関数の準備ができました。


In [10]:
# ========== セル 6: STEP 3-2 (テスト実行) ==========
# 目的: STEP 3-1で準備したDataFrameと関数を使って、少数の動画でダウンロードと文字起こしをテストします。

print("\n▶️ STEP 3-2: ダウンロード＆文字起こしのテスト実行を開始します...")

# 必要な変数が準備できているか確認
if 'df_step3_videos' not in globals() or not isinstance(df_step3_videos, pd.DataFrame) or df_step3_videos.empty:
    print("‼️エラー: テスト対象のDataFrame (df_step3_videos) が空か未定義です。「STEP 3-1」を先に実行してください。")
elif not all(var in globals() and globals()[var] is not None for var in ['MP3_DIR', 'TRANS_DIR', 'whisper_model', 'download_and_transcribe']):
    print("‼️エラー: 必要な変数/関数 (MP3_DIR, TRANS_DIR, whisper_model, download_and_transcribe) が未定義またはNoneです。")
    print("   「Imports and Constants」セルおよび「STEP 3-1」セルが正しく実行されているか確認してください。")
else:
    num_videos_to_test = 2  # テストする動画の件数を指定 (適宜変更してください)
    if len(df_step3_videos) < num_videos_to_test: # もし動画候補が指定件数より少なければ全件テスト
        num_videos_to_test = len(df_step3_videos)
        if num_videos_to_test > 0:
            print(f"   ℹ️ 動画候補が指定件数より少ないため、テスト件数を {num_videos_to_test} 件に変更します。")

    if num_videos_to_test == 0:
        print("   ℹ️ テスト対象の動画がありません。")
    else:
        print(f"--- テスト対象 (最初の {num_videos_to_test} 件の動画) ---")
        successful_tests = 0
        for i in range(num_videos_to_test):
            current_test_row = df_step3_videos.iloc[i]
            video_id_log = str(current_test_row.get('videoId', '不明なID'))
            video_title_log = str(current_test_row.get('title', '不明なタイトル'))
            print(f"テスト {i+1}/{num_videos_to_test}: Video ID [{video_id_log}], Title: '{video_title_log[:60]}...'")

            # download_and_transcribe 関数を呼び出し
            processed_id = download_and_transcribe(current_test_row, MP3_DIR, TRANS_DIR, whisper_model)

            if processed_id:
                print(f"   👍 テスト処理完了/スキップ: {processed_id}")
                successful_tests += 1
            else:
                print(f"   👎 テスト処理失敗: Video ID {video_id_log}")
            print("-" * 40) # 各テストの区切り

        print(f"--- テスト実行完了: {successful_tests} / {num_videos_to_test} 件のテスト処理が呼び出されました。 ---")
        print("   (個別のダウンロードや文字起こしの成否、エラー詳細は上記のログで確認してください)")

print("✅ STEP 3-2: テスト実行 完了。")


▶️ STEP 3-2: ダウンロード＆文字起こしのテスト実行を開始します...
--- テスト対象 (最初の 2 件の動画) ---
テスト 1/2: Video ID [h79B1Z4Bwyw], Title: 'Exposing our BESTSELLING Product (the exact strategy)...'
      ℹ️ ダウンロードスキップ: h79B1Z4Bwyw.mp3 は既に存在します。
      ℹ️ 文字起こしスキップ: h79B1Z4Bwyw.txt は既に存在します。
   👍 テスト処理完了/スキップ: h79B1Z4Bwyw
----------------------------------------
テスト 2/2: Video ID [G3fjxgYCct0], Title: '9 valuable lessons learned from the top business books...'
      ℹ️ ダウンロードスキップ: G3fjxgYCct0.mp3 は既に存在します。
      ℹ️ 文字起こしスキップ: G3fjxgYCct0.txt は既に存在します。
   👍 テスト処理完了/スキップ: G3fjxgYCct0
----------------------------------------
--- テスト実行完了: 2 / 2 件のテスト処理が呼び出されました。 ---
   (個別のダウンロードや文字起こしの成否、エラー詳細は上記のログで確認してください)
✅ STEP 3-2: テスト実行 完了。


In [11]:
# ========== セル 7: STEP 3-3 (本番実行) ==========
# 目的: STEP 3-1で準備したDataFrameと関数を使って、リストにある全ての動画のダウンロードと文字起こしを行います。

print("\n▶️ STEP 3-3: ダウンロード＆文字起こしの本番実行を開始します...")

# 必要な変数が準備できているか確認
if 'df_step3_videos' not in globals() or not isinstance(df_step3_videos, pd.DataFrame) or df_step3_videos.empty:
    print("‼️エラー: 処理対象のDataFrame (df_step3_videos) が空か未定義です。「STEP 3-1」を先に実行してください。")
elif not all(var in globals() and globals()[var] is not None for var in ['MP3_DIR', 'TRANS_DIR', 'whisper_model', 'download_and_transcribe']):
    print("‼️エラー: 必要な変数/関数 (MP3_DIR, TRANS_DIR, whisper_model, download_and_transcribe) が未定義またはNoneです。")
    print("   「Imports and Constants」セルおよび「STEP 3-1」セルが正しく実行されているか確認してください。")
else:
    total_video_count = len(df_step3_videos)
    current_processing_count = 0
    successful_processing_count = 0

    if total_video_count == 0: # DataFrameは存在するが中身が0件の場合
        print("ℹ️ 処理対象の動画が0件です。本番処理はスキップします。")
    else:
        print(f"--- 全 {total_video_count} 件の動画の処理を開始します ---")
        for index, video_row_to_process in df_step3_videos.iterrows(): # DataFrameの各行を処理
            current_processing_count += 1
            video_id_log_main = str(video_row_to_process.get('videoId', '不明なID'))
            video_title_log_main = str(video_row_to_process.get('title', '不明なタイトル'))
            print(f"処理中 {current_processing_count}/{total_video_count}: Video ID [{video_id_log_main}], Title: '{video_title_log_main[:60]}...'")

            # download_and_transcribe 関数を呼び出し
            result_video_id = download_and_transcribe(video_row_to_process, MP3_DIR, TRANS_DIR, whisper_model)

            if result_video_id:
                print(f"   👍 処理完了/スキップ: {result_video_id}")
                successful_processing_count += 1
            else:
                print(f"   👎 処理失敗: Video ID {video_id_log_main}")
            print("-" * 40) # 各動画処理の区切り

        print(f"--- 全件処理完了: {successful_processing_count} / {total_video_count} 件の処理が呼び出されました。---")
        print("    (個別のダウンロードや文字起こしの成否、エラー詳細は上記のログで確認してください)")

print("✅ STEP 3-3: 本番実行 が完了しました。")


▶️ STEP 3-3: ダウンロード＆文字起こしの本番実行を開始します...
--- 全 8 件の動画の処理を開始します ---
処理中 1/8: Video ID [h79B1Z4Bwyw], Title: 'Exposing our BESTSELLING Product (the exact strategy)...'
      ℹ️ ダウンロードスキップ: h79B1Z4Bwyw.mp3 は既に存在します。
      ℹ️ 文字起こしスキップ: h79B1Z4Bwyw.txt は既に存在します。
   👍 処理完了/スキップ: h79B1Z4Bwyw
----------------------------------------
処理中 2/8: Video ID [G3fjxgYCct0], Title: '9 valuable lessons learned from the top business books...'
      ℹ️ ダウンロードスキップ: G3fjxgYCct0.mp3 は既に存在します。
      ℹ️ 文字起こしスキップ: G3fjxgYCct0.txt は既に存在します。
   👍 処理完了/スキップ: G3fjxgYCct0
----------------------------------------
処理中 3/8: Video ID [lDwwLI1HQX8], Title: 'The Most Essential Traits for Successful Entrepreneurs with ...'
      ▶️ ダウンロード開始: lDwwLI1HQX8.mp3 (URL: https://youtu.be/lDwwLI1HQX8)
      ✅ ダウンロード完了: lDwwLI1HQX8.mp3
      ▶️ 文字起こし開始: lDwwLI1HQX8.txt (言語: en)
      ✅ 文字起こし完了: lDwwLI1HQX8.txt
   👍 処理完了/スキップ: lDwwLI1HQX8
----------------------------------------
処理中 4/8: Video ID [ahy9Ctcn5Ko], Title: '3 Daily Habit

## 5. Step 4: GPT Summarization & Script Generation

In [12]:
# ========== セル 8: STEP 4 (Gemini APIによる台本生成) ==========
# 目的: STEP3で作成された文字起こしテキストファイル (transcripts/フォルダ内) を元に、
#       Gemini APIを使って、MayaとShibaのキャラクターが対話するポッドキャスト台本を生成します。
#       生成された台本は scripts/フォルダに保存されます。

import time # APIリクエスト間の待機時間用

print("▶️ STEP 4: Gemini APIによる台本生成 を開始します...")

# --- 必要なグローバル変数が「Imports and Constants」セルで定義・設定されているか確認 ---
required_globals_step4 = ['DRIVE_ROOT', 'TRANS_DIR', 'SCRIPTS_DIR', 'gemini_api_configured']
missing_globals_step4 = [var for var in required_globals_step4 if var not in globals() or globals()[var] is None or (isinstance(globals()[var], bool) and not globals()[var])]
if missing_globals_step4:
    print(f"‼️エラー: STEP4に必要なグローバル変数が未定義、None、または設定されていません: {', '.join(missing_globals_step4)}")
    print("   「Imports and Constants」セルが正しく実行されているか確認してください。")
    # exit() # 処理を続行できない場合はここで終了

# --- Gemini モデル設定 ---
#
gemini_model_name_for_script = "gemini-2.0-flash"

gemini_script_model = None # Geminiモデルのインスタンスを格納する変数
if 'gemini_api_configured' in globals() and gemini_api_configured: # Gemini APIが設定済みの場合のみ
    try:
        print(f"   ℹ️ Geminiモデル ({gemini_model_name_for_script}) のインスタンス化を試みます...")
        gemini_script_model = genai.GenerativeModel(gemini_model_name_for_script)
        # 簡単なテストでモデルが応答するか確認 (任意)
        # test_response = gemini_script_model.generate_content("Hello Gemini!", generation_config=genai.types.GenerationConfig(max_output_tokens=10))
        # print(f"      Geminiモデルテスト応答: {test_response.text[:20]}...")
        print(f"   ✅ Geminiモデル ({gemini_model_name_for_script}) のインスタンス化に成功しました。")
    except Exception as e_gemini_model:
        print(f"‼️エラー: 指定されたGeminiモデル ({gemini_model_name_for_script}) のインスタンス化に失敗: {e_gemini_model}")
        print(f"   モデル名が正しいか、APIキーがこのモデルへのアクセス権を持っているか確認してください。")
else:
    print("‼️エラー: Gemini APIが設定されていないため、台本生成モデルを初期化できません。")

# Gemini APIの安全性設定 (コンテンツブロックを避けるため、必要に応じて調整)
gemini_safety_settings = [
    {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_NONE"},
]

# Gemini APIの生成設定 (出力の多様性などを調整)
gemini_generation_config = genai.types.GenerationConfig(
    temperature=0.7, # 0.0 (より決定的) ～ 1.0 (より創造的)。0.7はバランスが良い。
    # max_output_tokens=8000, # 出力トークンの最大数。モデルの制限とコストに注意。
)

# --- 文字起こしファイル (transcripts/*.txt) を処理 ---
scripts_generated_count = 0
scripts_failed_count = 0

# 必要なディレクトリとGeminiモデルが準備できているか最終確認
if not (DRIVE_ROOT and TRANS_DIR and SCRIPTS_DIR and os.path.isdir(TRANS_DIR)):
    print(f"‼️エラー: 必要なディレクトリ (TRANS_DIRなど) が設定されていないか、存在しません。台本生成は行えません。")
elif gemini_script_model is None: # Geminiモデルの準備に失敗した場合
    print(f"‼️エラー: Geminiモデルが初期化されていないため、台本生成は行えません。")
else:
    # transcriptsフォルダ内の全ての.txtファイルを取得し、名前順にソート
    all_transcript_files = sorted([f for f in os.listdir(TRANS_DIR) if f.endswith(".txt")])

    if not all_transcript_files:
        print(f"   ℹ️ transcripts フォルダ ({TRANS_DIR}) に処理対象の文字起こしファイル (.txt) が見つかりません。")
    else:
        print(f"   ▶️ {len(all_transcript_files)} 件の文字起こしファイルが見つかりました。台本生成を開始します。")

        for transcript_file_name_loop in all_transcript_files:
            video_id_from_filename = transcript_file_name_loop[:-4] # ファイル名から拡張子 ".txt" を除去してvideo_idとする
            current_transcript_path = os.path.join(TRANS_DIR, transcript_file_name_loop)
            # 生成する台本ファイルの名前は video_id.txt とする
            # (元のDL2Summary.ipynbでは _script.txt を付けていたが、シンプルにするため video_id.txt に)
            output_script_path = os.path.join(SCRIPTS_DIR, f"{video_id_from_filename}.txt")

            # 既に台本ファイルが存在する場合はスキップ
            if os.path.exists(output_script_path):
                print(f"   ⏭️ スキップ: 台本ファイル {output_script_path} は既に存在します。")
                scripts_generated_count += 1 # スキップも処理済みとしてカウント
                continue

            print(f"\n   処理中: {transcript_file_name_loop} (Video ID: {video_id_from_filename})")
            try:
                with open(current_transcript_path, 'r', encoding='utf-8') as f_transcript:
                    original_transcript_text = f_transcript.read()

                if not original_transcript_text.strip(): # 文字起こし内容が空ならスキップ
                    print(f"   ⚠️ 文字起こしファイル ({transcript_file_name_loop}) の内容が空です。台本生成をスキップします。")
                    scripts_failed_count += 1
                    continue

                # Gemini APIに渡すプロンプトを組み立てる
                # (前回の回答で改善提案した、Mayaの考察やShibaのリアクション多様化を盛り込んだプロンプト)
                prompt_text_for_gemini = f"""あなたは、日本のビジネスパーソン（特に"意識高い系"と呼ばれる層）に向けた、海外の有益な情報を解説するポッドキャストの台本作家です。
以下の文字起こし情報（これは海外の動画のものです）を元に、MayaとShibaの二人が対話する形式のポッドキャスト台本を日本語で作成してください。

# キャラクター設定
- Maya: 優しく落ち着いた口調の大人のお姉さん。リスナーを包み込むように、専門的な内容も分かりやすく解説します。敬語を使い、丁寧な言葉遣いを心がけます。**元の動画の内容を踏まえつつ、Maya自身の経験や他のビジネス事例、あるいは一般的なビジネス原則と結びつけて、独自の洞察や考察を付け加えてください。ただし、あくまで元動画の趣旨から大きく外れない範囲でお願いします。日本のビジネスパーソンが明日から使える具体的なアクションや、日本の状況に置き換えた場合の考え方も含められると尚良いです。**
- Shiba: 元気いっぱいで好奇心旺盛な女の子の柴犬。素直で、リスナーの代弁者のように「それってどういうこと？」「わぁ、すごい！」といった反応をします。時々「わん！」「くぅ～ん」といった可愛らしい相槌や感嘆詞を自然に入れますが、会話の流れを重視し、不自然にならないようにしてください。少し舌足らずな愛嬌のある話し方をします。**時には鋭いツッコミを入れたり、子供ならではの素朴な視点から核心を突く質問をしたりすることもあります。**

# 台本作成の指示
- 全体を通して、Mayaがメインの解説者、Shibaが聞き役兼リアクション役として、自然な会話の流れになるようにしてください。
- 文字起こし内容の**核心的なメッセージや重要なポイントをいくつか（できれば3つ程度）**選び出し、それらを軸に会話を展開してください。
- ポッドキャストの構成は以下を参考にしてください:
    1. **導入:**
        - Maya: リスナーへの挨拶と、今日のテーマ（元動画の主題）の簡単な紹介。
        - Shiba: 元気な挨拶と、テーマに対する興味を示すリアクション（例：「わーい！Mayaお姉ちゃん、今日はどんなお話？わくわく！」）。
    2. **本編 (各要点):**
        - Maya: 各要点を提示し、分かりやすく解説。**ここで独自の考察や日本の状況への言及を交えてください。**
        - Shiba: 各要点に対して、素朴な疑問を投げかけたり、感心したり、リスナーが共感できるようなコメントをします。（例：「へぇ～、それって日本の会社でも使えるのかなぁ？」「くぅん、なるほどね！それってつまり、〇〇ってこと？」）
    3. **リスナーへの実践的アドバイス:**
        - Maya: 今日の内容を踏まえ、リスナーが明日から試せる具体的な行動のヒントを1つ提示します。
        - Shiba: 「それならShibaにもできそう！わん！」のように、前向きな反応で後押しします。
    4. **まとめとクロージング:**
        - Shiba: 今日の話を聞いた感想や、一番心に残ったことを元気に述べます。
        - Maya: Shibaの感想を受け止め、リスナーへの感謝の言葉と、次回のポッドキャストへの期待感を込めたメッセージで締めくくります。
- 各セリフの行頭には、必ず話者を示すタグ「Maya: 」または「Shiba: 」を付けてください。
- 台本全体の長さは、日本語で800文字～1200文字程度を目安にしてください。
- 元の動画の言語（英語やスペイン語）に触れる必要はありません。あくまで日本語での解説コンテンツです。

# 文字起こし情報 (海外動画の内容):
---
{original_transcript_text}
---

上記に基づいて、魅力的なポッドキャスト台本を作成してください。
"""

                # Gemini APIを呼び出して台本を生成
                print(f"      💬 Gemini API ({gemini_model_name_for_script}) にリクエスト送信中...")
                api_response = gemini_script_model.generate_content(
                    prompt_text_for_gemini,
                    generation_config=gemini_generation_config,
                    safety_settings=gemini_safety_settings
                )
                generated_podcast_script = api_response.text # 生成されたテキストを取得

                # 生成された台本をファイルに保存
                with open(output_script_path, "w", encoding="utf-8") as f_script_out:
                    f_script_out.write(generated_podcast_script)
                print(f"   ✅ 台本作成完了: {output_script_path}")
                scripts_generated_count += 1

            except Exception as e_script_gen: # 台本生成中のエラー
                print(f"   ‼️エラー ({transcript_file_name_loop}): 台本生成中にエラーが発生: {e_script_gen}")
                # APIからの詳細なエラーフィードバックがあれば表示
                if 'api_response' in locals() and hasattr(api_response, 'prompt_feedback') and api_response.prompt_feedback:
                    print(f"      Gemini APIからのフィードバック: {api_response.prompt_feedback}")
                scripts_failed_count += 1
            finally:
                # APIのレート制限を考慮し、各ファイル処理後に少し待機
                # プレビュー版モデルや無料枠の場合は、もう少し長めに待つ(例: 5秒)ことを検討
                time.sleep(3) # 3秒待機 (状況に応じて調整)

        print(f"\n--- STEP 4 台本生成処理 全ファイル完了 ---")
        print(f"   処理成功 (またはスキップ): {scripts_generated_count} 件")
        if scripts_failed_count > 0:
            print(f"   処理失敗: {scripts_failed_count} 件")
        print("-" * 40)

print("✅ STEP 4: Gemini APIによる台本生成 が完了しました。")

▶️ STEP 4: Gemini APIによる台本生成 を開始します...
   ℹ️ Geminiモデル (gemini-2.0-flash) のインスタンス化を試みます...
   ✅ Geminiモデル (gemini-2.0-flash) のインスタンス化に成功しました。
   ▶️ 15 件の文字起こしファイルが見つかりました。台本生成を開始します。
   ⏭️ スキップ: 台本ファイル /content/drive/MyDrive/Colab Notebooks/00_Podcast/yt_podcast/scripts/433nP6kGRbQ.txt は既に存在します。
   ⏭️ スキップ: 台本ファイル /content/drive/MyDrive/Colab Notebooks/00_Podcast/yt_podcast/scripts/G3fjxgYCct0.txt は既に存在します。
   ⏭️ スキップ: 台本ファイル /content/drive/MyDrive/Colab Notebooks/00_Podcast/yt_podcast/scripts/MLu6Lf7Mg58.txt は既に存在します。
   ⏭️ スキップ: 台本ファイル /content/drive/MyDrive/Colab Notebooks/00_Podcast/yt_podcast/scripts/W2FjF450eZA.txt は既に存在します。
   ⏭️ スキップ: 台本ファイル /content/drive/MyDrive/Colab Notebooks/00_Podcast/yt_podcast/scripts/ahy9Ctcn5Ko.txt は既に存在します。
   ⏭️ スキップ: 台本ファイル /content/drive/MyDrive/Colab Notebooks/00_Podcast/yt_podcast/scripts/aiQj19TUMYM.txt は既に存在します。
   ⏭️ スキップ: 台本ファイル /content/drive/MyDrive/Colab Notebooks/00_Podcast/yt_podcast/scripts/h79B1Z4Bwyw.txt は既に存在します。

   処理中: hLBUuQpTgh